In [1]:
import pandas as pd
import numpy as np
import zipfile

In [2]:
import os
os.getcwd()

'C:\\Files\\Jupyter\\IoT\\赛题二-初赛'

In [3]:
os.chdir('C:\\Files\\Jupyter\\IoT\\赛题二-初赛')

In [9]:
# 检查train_data的Phase变化是否相同
variables1 = ['Phase']
TRAIN_ZIP=zipfile.ZipFile('.\Train\传感器高频数据.zip')
file_list=TRAIN_ZIP.namelist()

mode_data_Phase = {}

for i, f in enumerate(file_list):
    df = pd.read_csv(TRAIN_ZIP.open(f))
    mold_id = f.split('_')[3].replace('.csv', '')
    Phase = str(set(df['Phase']))
    mode_data_Phase[Phase] = mold_id

In [27]:
mode_data_Phase

{'{1.0, 3.0, 4.0, 6.0, 7.0, 8.0, 10.0, 11.0, 12.0, 14.0, 18.0}': '48895',
 '{1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 18}': '150716',
 '{0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 18}': '82774',
 '{1, 3, 4, 6, 7, 8, 10, 11, 12, 18}': '148466'}

In [49]:
# 训练集同一模式

# 检查Test_data的Phase变化是否相同
variables1 = ['Phase']
TEST_ZIP=zipfile.ZipFile('.\Test\传感器高频数据.zip')
file_list=TEST_ZIP.namelist()

mode_data_Phase = {}

for i, f in enumerate(file_list):
    df = pd.read_csv(TEST_ZIP.open(f))
    mold_id = f.split('_')[3].replace('.csv', '')
    Phase = str(set(df['Phase']))
    mode_data_Phase[Phase] = mold_id

In [50]:
mode_data_Phase

{'{1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 18}': '176241'}

# 比对合格/残次 与 流程异常的关系

In [51]:
Labels = pd.read_csv('.\Train\Size.csv',index_col=None)

In [52]:
Labels

,size1,size2,size3,Id
0,299.998,199.995,199.967,33722
1,299.999,199.938,199.956,34803
2,299.958,199.916,199.947,36621
3,299.974,199.930,199.961,36622
4,299.959,199.915,199.949,36623
...,...,...,...,...
16595,300.141,200.191,200.091,150696
16596,300.150,200.166,200.091,150701
16597,300.137,200.171,200.083,150706
16598,300.148,200.186,200.103,150711


In [65]:
flag1 = Labels['size1']<=300.15
flag2 = Labels['size1']>=299.85
flag3 = Labels['size2']<=200.075
flag4 = Labels['size2']>=199.925
flag5 = Labels['size3']<=200.075
flag6 = Labels['size3']>=199.925

In [73]:
flag = flag1 & flag2 & flag3 & flag4 & flag5 & flag6

In [80]:
passed = np.array(Labels[flag]['Id'])
failed = np.array(Labels[flag==False]['Id'])

In [151]:
len(passed),len(failed)

(13370, 3230)

In [92]:
routing1 = {1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 18}
routing2 = {0, 1, 3, 4, 6, 7, 8, 10, 11, 12, 14, 18}
routing3 = {1, 3, 4, 6, 7, 8, 10, 11, 12, 18}

In [93]:
# 检查train_data的Phase变化是否相同
TRAIN_ZIP=zipfile.ZipFile('.\Train\传感器高频数据.zip')
file_list=TRAIN_ZIP.namelist()

Routing1_ID = []
Routing2_ID = []
Routing3_ID = []

for i, f in enumerate(file_list):
    df = pd.read_csv(TRAIN_ZIP.open(f))
    mold_id = f.split('_')[3].replace('.csv', '')
    Phase = set(df['Phase'])
    if Phase == routing1:
        Routing1_ID.append(mold_id)
    elif Phase == routing2:
        Routing2_ID.append(mold_id)
    else:
        Routing3_ID.append(mold_id)

In [94]:
len(Routing1_ID), len(Routing2_ID), len(Routing3_ID)

(16596, 2, 2)

In [95]:
# 大多数正常。。。。

In [96]:
# 按Phase提取特征

In [129]:
# 检查train_data的Phase变化是否相同
import torch

TRAIN_ZIP=zipfile.ZipFile('.\Train\传感器高频数据.zip')
file_list=TRAIN_ZIP.namelist()

mold_train_x = []

for i, f in enumerate(file_list):
    df = pd.read_csv(TRAIN_ZIP.open(f))
    mold_id = f.split('_')[3].replace('.csv', '')
    Phase = set(df['Phase'].astype(int))
    if Phase == routing1:
        mold_train_x.extend(torch.tensor(tmp1.groupby(['Phase']).mean().sort_values(by=['SampleTime']).iloc[:,1:].values))
    else:
        print(mold_id)

traindata_x = torch.stack(mold_train_x).view(-1,1,11,22)

49897
82774
82793
148466


In [150]:
Labels = pd.read_csv('.\Train\Size.csv',index_col=None)



IDS = [49897,82774,82793,148466]

Labels.drop(Labels[Labels['Id'].isin(IDS)].index)

train_y1 = torch.tensor(Labels['size1'].values).view(-1,1)
train_y2 = torch.tensor(Labels['size2'].values).view(-1,1)
train_y3 = torch.tensor(Labels['size3'].values).view(-1,1)